In [ ]:
import os
import sys

import boto3
import matplotlib.pyplot as plt

# Add the mlops directory to the PYTHONPATH
sys.path.insert(0, os.path.abspath(".."))

from aws.ec2 import available_zones, gpu_instance_types, spot_supported_instance_types
from ipywidgets import widgets

%load_ext autoreload
%autoreload 2

In [ ]:
session = boto3.session.Session()
ec2 = session.client("ec2")

# EC2 available locations

In [ ]:
regions = session.get_available_regions("ec2")
regions_dropdown = widgets.Dropdown(
    options=regions,
    value=session.region_name,
    description="Region:",
)
regions_dropdown

In [ ]:
# Note that ec2 client is reassigned
ec2 = session.client("ec2", region_name=regions_dropdown.value)
zones = available_zones(ec2)
zones_dropdown = widgets.Dropdown(
    options=zones,
    description="Zone:",
)
zones_dropdown

# Instance type offerings

See [get_instance_types_from_instance_requirements](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.get_instance_types_from_instance_requirements), [Recommended GPU Instances](https://docs.aws.amazon.com/dlami/latest/devguide/gpu.html), and [Amazon EC2 Instance Types](https://aws.amazon.com/ec2/instance-types/) for more on instances and GPU offerings

In [ ]:
types = spot_supported_instance_types(ec2, gpu_instance_types(ec2))
types_dropdown = widgets.Dropdown(
    options=types,
    value="p2.xlarge" if "p2.xlarge" in types else types[0],
    description="Type:",
)
types_dropdown

# Pricing

In [ ]:
response = ec2.describe_spot_price_history(
    AvailabilityZone=zones_dropdown.value,
    InstanceTypes=[types_dropdown.value],
    MaxResults=50,
)

spot_price_history = response["SpotPriceHistory"]
prices = [float(item["SpotPrice"]) for item in spot_price_history]
time_stamps = [item["Timestamp"].isoformat() for item in spot_price_history]

plt.xticks(rotation=90)
plt.plot(time_stamps, prices)